# IMPORTS

In [ ]:
%load_ext autoreload
import os, sys
import numpy as np
import CL_inference as cl_inference

from bacco.baryons import emu_pars

%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('default')
plt.close('all')

font, rcnew = cl_inference.plot_utils.matplotlib_default_config()
mpl.rc('font', **font)
plt.rcParams.update(rcnew)
plt.style.use('tableau-colorblind10')
%config InlineBackend.figure_format = 'retina'

# GENERATE DATASETS

In [ ]:
COSMOS = {}
XX = {}
AUG_PARAMS = {}

In [ ]:
kmax = 0.6
path_save_root = os.path.join("/cosmos_storage/dlopez/Projects/CL_inference/DATASETS", "DATASET_kmax_"+str(kmax))

### sample (latin hypercube) fixed points in baryon param space

In [ ]:
NN_fixed_latin = 10
dict_bounds_augs = dict(
    M_c       = [9.0, 15.0],
    eta       = [-0.69, 0.69],
    beta      = [-1.00, 0.69],
    M1_z0_cen = [9.0, 13.0],
    theta_out = [0., 0.47],
    theta_inn = [-2.0, -0.523],
    M_inn     = [9.0, 13.5]
)
sample_fixed_baryon_models = cl_inference.preprocessing_tools.sample_latin_hypercube(
    dict_bounds_augs, N_points=NN_fixed_latin, seed=0
)

### TRAIN

In [ ]:
ds_mode = 'TRAIN'
COSMOS[ds_mode] = {}
XX[ds_mode] = {}
AUG_PARAMS[ds_mode] = {}

NN_samples_cosmo = 8192
seed = 0

path_save = os.path.join(path_save_root, ds_mode)

dict_bounds_cosmo = dict(
    omega_cold    = [0.24, 0.39],
    omega_baryon  = [0.041, 0.059],
    hubble        = [0.61, 0.79],
    ns            = [0.93, 1.0],
    sigma8_cold   = [0.74, 0.89],
    expfactor     = 1.,
    neutrino_mass = 0.,
    w0            = -1.,
    wa            = 0.
)

model_name = "Model_fixed_nl"
mode_baccoemu = "nonlinear"
NN_samples_augs = 1
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    mode_baccoemu     = mode_baccoemu,
    kmax              = kmax
)

model_name = "Model_vary_all"
NN_samples_augs = 20
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

# model_name = "Model_fixed_1"
# NN_samples_augs = 1
# dict_bounds_augs = dict(M_c=11.5, eta=-0.6, beta=0.0, M1_z0_cen=11.5, theta_out=0., theta_inn=-1., M_inn=12.)
# COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
#     NN_samples_cosmo  = NN_samples_cosmo,
#     NN_samples_augs   = NN_samples_augs,
#     dict_bounds_cosmo = dict_bounds_cosmo,
#     dict_bounds_augs  = dict_bounds_augs,
#     seed              = seed,
#     path_save         = path_save,
#     model_name        = model_name,
#     kmax              = kmax
# )

# model_name = "Model_fixed_2"
# NN_samples_augs = 1
# dict_bounds_augs = dict(M_c=15., eta=-0.6, beta=0.0, M1_z0_cen=11.5, theta_out=0., theta_inn=-1., M_inn=12.)
# COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
#     NN_samples_cosmo  = NN_samples_cosmo,
#     NN_samples_augs   = NN_samples_augs,
#     dict_bounds_cosmo = dict_bounds_cosmo,
#     dict_bounds_augs  = dict_bounds_augs,
#     seed              = seed,
#     path_save         = path_save,
#     model_name        = model_name,
#     kmax              = kmax
# )

hydro_key = 'bahamas'
model_name = "Model_fixed_"+hydro_key
NN_samples_augs = 1
dict_bounds_augs = dict(
    M_c       = emu_pars(model=hydro_key)['z_0.0']['M_c'],
    eta       = emu_pars(model=hydro_key)['z_0.0']['eta'],
    beta      = emu_pars(model=hydro_key)['z_0.0']['beta'],
    M1_z0_cen = emu_pars(model=hydro_key)['z_0.0']['M1_z0_cen'],
    theta_out = emu_pars(model=hydro_key)['z_0.0']['theta_out'],
    theta_inn = emu_pars(model=hydro_key)['z_0.0']['theta_inn'],
    M_inn     = emu_pars(model=hydro_key)['z_0.0']['M_inn']
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

hydro_key = 'eagle'
model_name = "Model_fixed_"+hydro_key
NN_samples_augs = 1
dict_bounds_augs = dict(
    M_c       = emu_pars(model=hydro_key)['z_0.0']['M_c'],
    eta       = emu_pars(model=hydro_key)['z_0.0']['eta'],
    beta      = emu_pars(model=hydro_key)['z_0.0']['beta'],
    M1_z0_cen = emu_pars(model=hydro_key)['z_0.0']['M1_z0_cen'],
    theta_out = emu_pars(model=hydro_key)['z_0.0']['theta_out'],
    theta_inn = emu_pars(model=hydro_key)['z_0.0']['theta_inn'],
    M_inn     = emu_pars(model=hydro_key)['z_0.0']['M_inn']
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

hydro_key = 'illustris'
model_name = "Model_fixed_"+hydro_key
NN_samples_augs = 1
dict_bounds_augs = dict(
    M_c       = emu_pars(model=hydro_key)['z_0.0']['M_c'],
    eta       = emu_pars(model=hydro_key)['z_0.0']['eta'],
    beta      = emu_pars(model=hydro_key)['z_0.0']['beta'],
    M1_z0_cen = emu_pars(model=hydro_key)['z_0.0']['M1_z0_cen'],
    theta_out = emu_pars(model=hydro_key)['z_0.0']['theta_out'],
    theta_inn = emu_pars(model=hydro_key)['z_0.0']['theta_inn'],
    M_inn     = emu_pars(model=hydro_key)['z_0.0']['M_inn']
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

model_name = "Model_vary_1"
NN_samples_augs = 10
dict_bounds_augs = dict(
    M_c       = [9.0, 15.0],
    eta       = [-0.69, 0.69],
    beta      = 0.,
    M1_z0_cen = 12.,
    theta_out = 0.,
    theta_inn = -1.,
    M_inn     = 13.
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

model_name = "Model_vary_2"
NN_samples_augs = 10
dict_bounds_augs = dict(
    M_c       = 13.5,
    eta       = 0.,
    beta      = [-1.00, 0.69],
    M1_z0_cen = [9.0, 13.0],
    theta_out = 0.,
    theta_inn = -1.,
    M_inn     = 13.
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

model_name = "Model_vary_3"
NN_samples_augs = 10
dict_bounds_augs = dict(
    M_c       = 13.5,
    eta       = 0.,
    beta      = 0.,
    M1_z0_cen = 12.,
    theta_out = [0., 0.47],
    theta_inn = [-2.0, -0.523],
    M_inn     = [9.0, 13.5]
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

for ii in range(sample_fixed_baryon_models.shape[0]):
    model_name = "Model_fixed_" + str(ii)
    NN_samples_augs = 1
    dict_bounds_augs = dict(
        M_c       = sample_fixed_baryon_models[ii,0],
        eta       = sample_fixed_baryon_models[ii,1],
        beta      = sample_fixed_baryon_models[ii,2],
        M1_z0_cen = sample_fixed_baryon_models[ii,3],
        theta_out = sample_fixed_baryon_models[ii,4],
        theta_inn = sample_fixed_baryon_models[ii,5],
        M_inn     = sample_fixed_baryon_models[ii,6],
    )
    COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
        NN_samples_cosmo  = NN_samples_cosmo,
        NN_samples_augs   = NN_samples_augs,
        dict_bounds_cosmo = dict_bounds_cosmo,
        dict_bounds_augs  = dict_bounds_augs,
        seed              = seed,
        path_save         = path_save,
        model_name        = model_name,
        kmax              = kmax
    )

### VAL

In [ ]:
ds_mode = 'VAL'
COSMOS[ds_mode] = {}
XX[ds_mode] = {}
AUG_PARAMS[ds_mode] = {}

NN_samples_cosmo = 2048
seed = 1

path_save = os.path.join(path_save_root, ds_mode)

dict_bounds_cosmo = dict(
    omega_cold    = [0.24, 0.39],
    omega_baryon  = [0.041, 0.059],
    hubble        = [0.61, 0.79],
    ns            = [0.93, 1.0],
    sigma8_cold   = [0.74, 0.89],
    expfactor     = 1.,
    neutrino_mass = 0.,
    w0            = -1.,
    wa            = 0.
)

model_name = "Model_fixed_nl"
mode_baccoemu = "nonlinear"
NN_samples_augs = 1
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    mode_baccoemu     = mode_baccoemu,
    kmax              = kmax
)

model_name = "Model_vary_all"
NN_samples_augs = 20
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

# model_name = "Model_fixed_1"
# NN_samples_augs = 1
# dict_bounds_augs = dict(M_c=11.5, eta=-0.6, beta=0.0, M1_z0_cen=11.5, theta_out=0., theta_inn=-1., M_inn=12.)
# COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
#     NN_samples_cosmo  = NN_samples_cosmo,
#     NN_samples_augs   = NN_samples_augs,
#     dict_bounds_cosmo = dict_bounds_cosmo,
#     dict_bounds_augs  = dict_bounds_augs,
#     seed              = seed,
#     path_save         = path_save,
#     model_name        = model_name,
#     kmax              = kmax
# )

# model_name = "Model_fixed_2"
# NN_samples_augs = 1
# dict_bounds_augs = dict(M_c=15., eta=-0.6, beta=0.0, M1_z0_cen=11.5, theta_out=0., theta_inn=-1., M_inn=12.)
# COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
#     NN_samples_cosmo  = NN_samples_cosmo,
#     NN_samples_augs   = NN_samples_augs,
#     dict_bounds_cosmo = dict_bounds_cosmo,
#     dict_bounds_augs  = dict_bounds_augs,
#     seed              = seed,
#     path_save         = path_save,
#     model_name        = model_name,
#     kmax              = kmax
# )

hydro_key = 'bahamas'
model_name = "Model_fixed_"+hydro_key
NN_samples_augs = 1
dict_bounds_augs = dict(
    M_c       = emu_pars(model=hydro_key)['z_0.0']['M_c'],
    eta       = emu_pars(model=hydro_key)['z_0.0']['eta'],
    beta      = emu_pars(model=hydro_key)['z_0.0']['beta'],
    M1_z0_cen = emu_pars(model=hydro_key)['z_0.0']['M1_z0_cen'],
    theta_out = emu_pars(model=hydro_key)['z_0.0']['theta_out'],
    theta_inn = emu_pars(model=hydro_key)['z_0.0']['theta_inn'],
    M_inn     = emu_pars(model=hydro_key)['z_0.0']['M_inn']
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

hydro_key = 'eagle'
model_name = "Model_fixed_"+hydro_key
NN_samples_augs = 1
dict_bounds_augs = dict(
    M_c       = emu_pars(model=hydro_key)['z_0.0']['M_c'],
    eta       = emu_pars(model=hydro_key)['z_0.0']['eta'],
    beta      = emu_pars(model=hydro_key)['z_0.0']['beta'],
    M1_z0_cen = emu_pars(model=hydro_key)['z_0.0']['M1_z0_cen'],
    theta_out = emu_pars(model=hydro_key)['z_0.0']['theta_out'],
    theta_inn = emu_pars(model=hydro_key)['z_0.0']['theta_inn'],
    M_inn     = emu_pars(model=hydro_key)['z_0.0']['M_inn']
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

hydro_key = 'illustris'
model_name = "Model_fixed_"+hydro_key
NN_samples_augs = 1
dict_bounds_augs = dict(
    M_c       = emu_pars(model=hydro_key)['z_0.0']['M_c'],
    eta       = emu_pars(model=hydro_key)['z_0.0']['eta'],
    beta      = emu_pars(model=hydro_key)['z_0.0']['beta'],
    M1_z0_cen = emu_pars(model=hydro_key)['z_0.0']['M1_z0_cen'],
    theta_out = emu_pars(model=hydro_key)['z_0.0']['theta_out'],
    theta_inn = emu_pars(model=hydro_key)['z_0.0']['theta_inn'],
    M_inn     = emu_pars(model=hydro_key)['z_0.0']['M_inn']
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

model_name = "Model_vary_1"
NN_samples_augs = 10
dict_bounds_augs = dict(
    M_c       = [9.0, 15.0],
    eta       = [-0.69, 0.69],
    beta      = 0.,
    M1_z0_cen = 12.,
    theta_out = 0.,
    theta_inn = -1.,
    M_inn     = 13.
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

model_name = "Model_vary_2"
NN_samples_augs = 10
dict_bounds_augs = dict(
    M_c       = 13.5,
    eta       = 0.,
    beta      = [-1.00, 0.69],
    M1_z0_cen = [9.0, 13.0],
    theta_out = 0.,
    theta_inn = -1.,
    M_inn     = 13.
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

model_name = "Model_vary_3"
NN_samples_augs = 10
dict_bounds_augs = dict(
    M_c       = 13.5,
    eta       = 0.,
    beta      = 0.,
    M1_z0_cen = 12.,
    theta_out = [0., 0.47],
    theta_inn = [-2.0, -0.523],
    M_inn     = [9.0, 13.5]
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

for ii in range(sample_fixed_baryon_models.shape[0]):
    model_name = "Model_fixed_" + str(ii)
    NN_samples_augs = 1
    dict_bounds_augs = dict(
        M_c       = sample_fixed_baryon_models[ii,0],
        eta       = sample_fixed_baryon_models[ii,1],
        beta      = sample_fixed_baryon_models[ii,2],
        M1_z0_cen = sample_fixed_baryon_models[ii,3],
        theta_out = sample_fixed_baryon_models[ii,4],
        theta_inn = sample_fixed_baryon_models[ii,5],
        M_inn     = sample_fixed_baryon_models[ii,6],
    )
    COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
        NN_samples_cosmo  = NN_samples_cosmo,
        NN_samples_augs   = NN_samples_augs,
        dict_bounds_cosmo = dict_bounds_cosmo,
        dict_bounds_augs  = dict_bounds_augs,
        seed              = seed,
        path_save         = path_save,
        model_name        = model_name,
        kmax              = kmax
    )

### TEST

In [ ]:
ds_mode = 'TEST'
COSMOS[ds_mode] = {}
XX[ds_mode] = {}
AUG_PARAMS[ds_mode] = {}

NN_samples_cosmo = 2048
seed = 2

path_save = os.path.join(path_save_root, ds_mode)

dict_bounds_cosmo = dict(
    omega_cold    = [0.24, 0.39],
    omega_baryon  = [0.041, 0.059],
    hubble        = [0.61, 0.79],
    ns            = [0.93, 1.0],
    sigma8_cold   = [0.74, 0.89],
    expfactor     = 1.,
    neutrino_mass = 0.,
    w0            = -1.,
    wa            = 0.
)

model_name = "Model_fixed_nl"
mode_baccoemu = "nonlinear"
NN_samples_augs = 1
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    mode_baccoemu     = mode_baccoemu,
    kmax              = kmax
)

model_name = "Model_vary_all"
NN_samples_augs = 20
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

# model_name = "Model_fixed_1"
# NN_samples_augs = 1
# dict_bounds_augs = dict(M_c=11.5, eta=-0.6, beta=0.0, M1_z0_cen=11.5, theta_out=0., theta_inn=-1., M_inn=12.)
# COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
#     NN_samples_cosmo  = NN_samples_cosmo,
#     NN_samples_augs   = NN_samples_augs,
#     dict_bounds_cosmo = dict_bounds_cosmo,
#     dict_bounds_augs  = dict_bounds_augs,
#     seed              = seed,
#     path_save         = path_save,
#     model_name        = model_name,
#     kmax              = kmax
# )

# model_name = "Model_fixed_2"
# NN_samples_augs = 1
# dict_bounds_augs = dict(M_c=15., eta=-0.6, beta=0.0, M1_z0_cen=11.5, theta_out=0., theta_inn=-1., M_inn=12.)
# COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
#     NN_samples_cosmo  = NN_samples_cosmo,
#     NN_samples_augs   = NN_samples_augs,
#     dict_bounds_cosmo = dict_bounds_cosmo,
#     dict_bounds_augs  = dict_bounds_augs,
#     seed              = seed,
#     path_save         = path_save,
#     model_name        = model_name,
#     kmax              = kmax
# )

hydro_key = 'bahamas'
model_name = "Model_fixed_"+hydro_key
NN_samples_augs = 1
dict_bounds_augs = dict(
    M_c       = emu_pars(model=hydro_key)['z_0.0']['M_c'],
    eta       = emu_pars(model=hydro_key)['z_0.0']['eta'],
    beta      = emu_pars(model=hydro_key)['z_0.0']['beta'],
    M1_z0_cen = emu_pars(model=hydro_key)['z_0.0']['M1_z0_cen'],
    theta_out = emu_pars(model=hydro_key)['z_0.0']['theta_out'],
    theta_inn = emu_pars(model=hydro_key)['z_0.0']['theta_inn'],
    M_inn     = emu_pars(model=hydro_key)['z_0.0']['M_inn']
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

hydro_key = 'eagle'
model_name = "Model_fixed_"+hydro_key
NN_samples_augs = 1
dict_bounds_augs = dict(
    M_c       = emu_pars(model=hydro_key)['z_0.0']['M_c'],
    eta       = emu_pars(model=hydro_key)['z_0.0']['eta'],
    beta      = emu_pars(model=hydro_key)['z_0.0']['beta'],
    M1_z0_cen = emu_pars(model=hydro_key)['z_0.0']['M1_z0_cen'],
    theta_out = emu_pars(model=hydro_key)['z_0.0']['theta_out'],
    theta_inn = emu_pars(model=hydro_key)['z_0.0']['theta_inn'],
    M_inn     = emu_pars(model=hydro_key)['z_0.0']['M_inn']
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

hydro_key = 'illustris'
model_name = "Model_fixed_"+hydro_key
NN_samples_augs = 1
dict_bounds_augs = dict(
    M_c       = emu_pars(model=hydro_key)['z_0.0']['M_c'],
    eta       = emu_pars(model=hydro_key)['z_0.0']['eta'],
    beta      = emu_pars(model=hydro_key)['z_0.0']['beta'],
    M1_z0_cen = emu_pars(model=hydro_key)['z_0.0']['M1_z0_cen'],
    theta_out = emu_pars(model=hydro_key)['z_0.0']['theta_out'],
    theta_inn = emu_pars(model=hydro_key)['z_0.0']['theta_inn'],
    M_inn     = emu_pars(model=hydro_key)['z_0.0']['M_inn']
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

model_name = "Model_vary_1"
NN_samples_augs = 10
dict_bounds_augs = dict(
    M_c       = [9.0, 15.0],
    eta       = [-0.69, 0.69],
    beta      = 0.,
    M1_z0_cen = 12.,
    theta_out = 0.,
    theta_inn = -1.,
    M_inn     = 13.
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

model_name = "Model_vary_2"
NN_samples_augs = 10
dict_bounds_augs = dict(
    M_c       = 13.5,
    eta       = 0.,
    beta      = [-1.00, 0.69],
    M1_z0_cen = [9.0, 13.0],
    theta_out = 0.,
    theta_inn = -1.,
    M_inn     = 13.
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

model_name = "Model_vary_3"
NN_samples_augs = 10
dict_bounds_augs = dict(
    M_c       = 13.5,
    eta       = 0.,
    beta      = 0.,
    M1_z0_cen = 12.,
    theta_out = [0., 0.47],
    theta_inn = [-2.0, -0.523],
    M_inn     = [9.0, 13.5]
)
COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo  = NN_samples_cosmo,
    NN_samples_augs   = NN_samples_augs,
    dict_bounds_cosmo = dict_bounds_cosmo,
    dict_bounds_augs  = dict_bounds_augs,
    seed              = seed,
    path_save         = path_save,
    model_name        = model_name,
    kmax              = kmax
)

for ii in range(sample_fixed_baryon_models.shape[0]):
    model_name = "Model_fixed_" + str(ii)
    NN_samples_augs = 1
    dict_bounds_augs = dict(
        M_c       = sample_fixed_baryon_models[ii,0],
        eta       = sample_fixed_baryon_models[ii,1],
        beta      = sample_fixed_baryon_models[ii,2],
        M1_z0_cen = sample_fixed_baryon_models[ii,3],
        theta_out = sample_fixed_baryon_models[ii,4],
        theta_inn = sample_fixed_baryon_models[ii,5],
        M_inn     = sample_fixed_baryon_models[ii,6],
    )
    COSMOS[ds_mode][model_name], XX[ds_mode][model_name], AUG_PARAMS[ds_mode][model_name] = cl_inference.preprocessing_tools.generate_baccoemu_dataset(
        NN_samples_cosmo  = NN_samples_cosmo,
        NN_samples_augs   = NN_samples_augs,
        dict_bounds_cosmo = dict_bounds_cosmo,
        dict_bounds_augs  = dict_bounds_augs,
        seed              = seed,
        path_save         = path_save,
        model_name        = model_name,
        kmax              = kmax
    )

# VALIDATE DATASETS

In [ ]:
ds_mode = 'TEST'
model_name = 'Model_vary_2'

cosmos = COSMOS[ds_mode][model_name]
xx = XX[ds_mode][model_name]
aug_params = AUG_PARAMS[ds_mode][model_name]

print(cosmos.shape)
print(np.min(cosmos, axis=0))
print(np.max(cosmos, axis=0))

print(xx.shape)
print(xx.min())
print(xx.max())

print(aug_params.shape)
tmp_aug_params = np.reshape(aug_params, (aug_params.shape[0]*aug_params.shape[1], aug_params.shape[-1]))
print(tmp_aug_params.shape)
print(np.min(tmp_aug_params, axis=0))
print(np.max(tmp_aug_params, axis=0))

In [ ]:
kmin=-2.3
N_kk = int(((kmax-kmin)/(0.6+2.3))*100)
kk = np.logspace(kmin, kmax, num=N_kk)

In [ ]:
ds_mode = 'TEST'
list_plot_models = [ "Model_fixed_nl"]
list_plot_models = [ "Model_fixed_1", "Model_fixed_2"]
list_plot_models = [ "Model_vary_1", "Model_vary_2"]
list_plot_models = [ "Model_vary_all"]

fig, axs = mpl.pyplot.subplots(2,1,figsize=(9,9), gridspec_kw={'height_ratios': [1.5, 1]})
axs[0].set_ylabel(r'$P(k) \left[ \left(h^{-1} \mathrm{Mpc}\right)^{3} \right]$')
axs[1].set_ylabel(r'$P_{Model}(k) / P_{nl}(k)$')
axs[1].set_xlabel(r'$\mathrm{Wavenumber}\, k \left[ h\, \mathrm{Mpc}^{-1} \right]$')
axs[0].set_title(ds_mode, fontsize=22)

N_plot_cosmo = 5
indexes_cosmo = np.random.choice(XX[ds_mode][model_name].shape[0], N_plot_cosmo, replace=False)
# print("indexes_cosmo", indexes_cosmo)

colors = cl_inference.plot_utils.get_N_colors(N_plot_cosmo, mpl.colormaps['prism'])
linestyles = cl_inference.plot_utils.get_N_linestyles(len(list_plot_models))
custom_lines_cosmo = []
custom_names_cosmo = []
custom_lines_augs = []
custom_names_augs = []
for ii_cosmo, index_cosmo in enumerate(indexes_cosmo):
    custom_lines_cosmo.append(mpl.lines.Line2D([0], [0], color=colors[ii_cosmo], ls='-', lw=10, marker=None, markersize=8))
    custom_names_cosmo.append('Cosmo # '+str(ii_cosmo))
    for ii_model, model_name in enumerate(list_plot_models):
        axs[0].plot(
            np.array(kk), 10**XX[ds_mode][model_name][ii_cosmo,:].T,
            c=colors[ii_cosmo], linestyle=linestyles[ii_model], lw=1, marker=None, ms=2
        )
        axs[1].plot(
            np.array(kk),
            10**(XX[ds_mode][model_name][ii_cosmo,:].T)/10**(XX[ds_mode]['Model_fixed_nl'][ii_cosmo,:].T),
            c=colors[ii_cosmo], linestyle=linestyles[ii_model], lw=1, marker=None, ms=2
         )
        if ii_cosmo == 0:
            custom_lines_augs.append(mpl.lines.Line2D([0], [0], color='grey', ls=linestyles[ii_model], lw=1, marker=None, markersize=8))
            custom_names_augs.append('Aug - '+model_name)

axs[0].axvline(10**kmax, c='k', ls=':', lw=1)
axs[1].axvline(10**kmax, c='k', ls=':', lw=1)
            
axs[0].set_xscale('log')
axs[1].set_xscale('log')
axs[0].set_yscale('log')
legend = axs[0].legend(custom_lines_cosmo, custom_names_cosmo, loc='lower left', fancybox=True, shadow=True, ncol=1,fontsize=16)
axs[0].add_artist(legend)
legend = axs[1].legend(custom_lines_augs, custom_names_augs, loc='upper left', fancybox=True, shadow=True, ncol=1,fontsize=12)
axs[1].add_artist(legend)
axs[0].set_xlim([0.004, 4.5])
axs[1].set_xlim([0.004, 4.5])
axs[0].set_ylim([40., 50000])
axs[1].set_ylim([0.78, 1.2])
axs[0].set_xticklabels([])
plt.tight_layout()
plt.show()

# Validate dataset loaders

In [ ]:
path_load = os.path.join(path_save_root, "TEST")
list_model_names = ['Model_vary_1', 'Model_vary_2']

loaded_theta, loaded_xx = cl_inference.data_tools.load_stored_data(
    path_load,
    list_model_names
)

In [ ]:
print(loaded_theta.shape)
print(loaded_xx.shape)

In [ ]:
normalize=False
path_save_norm = None
path_load_norm = None

NN_augs_batch = 2
add_noise_Pk="cosmic_var_gauss"
boxsize_cosmic_variance=1000 # Mpc/h
kmax=0.6

dset = cl_inference.data_tools.data_loader(
    loaded_theta,
    loaded_xx,
    normalize=normalize,
    path_save_norm=path_save_norm,
    path_load_norm=path_load_norm,
    NN_augs_batch=NN_augs_batch,
    add_noise_Pk=add_noise_Pk,
    boxsize_cosmic_variance=boxsize_cosmic_variance, # Mpc/h
    kmax=kmax
)

In [ ]:
print(dset.xx.shape)
print(dset.theta.shape)
print(dset.NN_augs_batch)

In [ ]:
batch_size = 3
seed = 0
return_indexes_sampled=True
indexes_cosmo=None
indexes_augs=None
use_all_dataset_augs_ordered=False
to_torch=False
device="cpu"

theta_batch, xx_batch, indexes_cosmo, indexes_augs = dset(
    batch_size=batch_size,
    seed=seed,
    return_indexes_sampled=return_indexes_sampled,
    indexes_cosmo=indexes_cosmo,
    indexes_augs=indexes_augs,
    use_all_dataset_augs_ordered=use_all_dataset_augs_ordered,
    to_torch=to_torch,
    device=device
)

In [ ]:
list_corresponding_model_names = []
list_corresponding_aug_index = []
for ii_key, key in enumerate(list_model_names):
    list_corresponding_model_names.extend([key]*XX[ds_mode][key].shape[1])
    list_corresponding_aug_index.extend(np.arange(XX[ds_mode][key].shape[1]))

In [ ]:
fig, ax = mpl.pyplot.subplots(1,1,figsize=(7,6))
ax.set_ylabel(r'$P(k) \left[ \left(h^{-1} \mathrm{Mpc}\right)^{3} \right]$')
ax.set_xlabel(r'$\mathrm{Wavenumber}\, k \left[ h\, \mathrm{Mpc}^{-1} \right]$')

colors = cl_inference.plot_utils.get_N_colors(batch_size*NN_augs_batch, mpl.colormaps['jet'])
for ii_cosmo, index_cosmo in enumerate(indexes_cosmo):
    for ii_aug, index_aug in enumerate(indexes_augs[ii_cosmo]):
        tmp_model_name = list_corresponding_model_names[index_aug]   
        tmp_aug_index = list_corresponding_aug_index[index_aug]
        ax.plot(
            np.array(kk), 10**XX[ds_mode][tmp_model_name][index_cosmo][tmp_aug_index],
            c=colors[ii_cosmo*NN_augs_batch+ii_aug], linestyle='-', lw=1., marker=None, ms=2
        )
        ax.plot(
            np.array(kk), 10**loaded_xx[index_cosmo][index_aug],
            c=colors[ii_cosmo*NN_augs_batch+ii_aug], linestyle=':', lw=1.5, marker=None, ms=2
        )
        ax.plot(
            np.array(kk), 10**dset.xx[index_cosmo][index_aug],
            c=colors[ii_cosmo*NN_augs_batch+ii_aug], linestyle='--', lw=2., marker=None, ms=2
        )
        ax.plot(
            np.array(kk), 10**xx_batch[ii_cosmo, ii_aug],
            c=colors[ii_cosmo*NN_augs_batch+ii_aug], linestyle=':', lw=2
            , marker=None, ms=2
        )
ax.axvline(10**kmax, c='k', ls=':', lw=1)        
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim([0.004, 4.5])
ax.set_ylim([40., 70000])
plt.tight_layout()
plt.show()

# Test use_all_dataset_augs_ordered mode=True and to_torch=True

In [ ]:
batch_size = 0
seed = 0
return_indexes_sampled=True
indexes_cosmo=None
indexes_augs=None
use_all_dataset_augs_ordered=True
to_torch=True
device="cpu"

theta_batch, xx_batch, indexes_cosmo, indexes_augs = dset(
    batch_size=batch_size,
    seed=seed,
    return_indexes_sampled=return_indexes_sampled,
    indexes_cosmo=indexes_cosmo,
    indexes_augs=indexes_augs,
    use_all_dataset_augs_ordered=use_all_dataset_augs_ordered,
    to_torch=to_torch,
    device=device
)

print(theta_batch.shape)
print(xx_batch.shape)